# Similarity Analysis among New York, Toronto and Hong Kong

## 1. Business Background

New York, Toronto and Hong Kong and famous cities in the world. It is worthy to have a similarity analysis among these cities to understand the different cultures of different nationalities. According to previous study and based on that, we would like to use Foursqaure data to provide the most popular type of venues in different neighborhoods for analysis. Where New York and Toronto are in North America but different countries, and Hong Kong is in Asia, we expect Toronto is more similar to New York than Hong Kong.

*(referring to Couresra Project Week 4 Part 1)*

## 2. Analytic Approach

Given that we have the geolocation data of New York and Toronto, we also prepare the data for Hong Kong referring to their post offices' addresses. This data is provided by https://geodata.gov.hk/.

Make use of the geolocations, it represents the neighborhoods of an area and we can use Foursqaure API to get the list of venues surrounding. In order to compare the cities, we will sum up the counts of common places of the city to prepare the top 10 common places to compare.

Down to the neighborhoods, we will also apply K-MEAN clustering to label the different neighborhoods to see if the classes are evenly distribute or not.

*(referring to Couresra Project Week 4 Part 2)*

## 3. Data Requirement

## 4. Data Collection